In [2]:
import numpy as np

# 関数一覧

## 流体計算
### 非粘性流束評価
$\triangle F_i^n=A(\triangle Q_{i}^n ) = A (Q_{i+\frac{1}{2}}^n - Q_{i-\frac{1}{2}}^n )$


### 物理式の評価
反復演算子$ L = 
\frac{1}{\triangle t} \left( 
        Q_i^{n+1}
        - \frac{ \triangle x_i^{n} }{ \triangle x_i^{n+1} } Q_i^{n}
    \right)
    + 
    \frac{1}{\triangle x_i^{n+1}} \left(
        F_{i + \frac{1}{2}}^{ n + \frac{1}{2}}
      - F_{i - \frac{1}{2}}^{ n + \frac{1}{2}}
    \right)
    - \frac{1}{\triangle t} \left(
        \frac{ \delta x_{i + \frac{1}{2}}^{ n + \frac{1}{2}} }
             {\triangle x_{i}^{ n + 1}}
        Q_{i + \frac{1}{2}}^{ n + \frac{1}{2}}
      - \frac{ \delta x_{i - \frac{1}{2}}^{ n + \frac{1}{2}} }
             {\triangle x_{i}^{ n + 1}}
        Q_{i - \frac{1}{2}}^{ n + \frac{1}{2}}
    \right)
    + \frac{1}{2} \left(
        1 + \frac{ \triangle x_i^{n} }{ \triangle x_i^{n+1} } 
    \right) \bar{S_i^{n+\frac{1}{2}}}
$

## 便利関数
無次元変数と有次元変数の相互変換を実施
有次元変数 = scaler_n2d(無次元変数)  
無次元変数 = scaler_d2n(有次元変数)  



Flux Jacobianについて

$Q = \left[
\begin{matrix}
\rho \\ 
\rho u \\
e
\end{matrix}
\right]$

$F = \left[
\begin{matrix}
\rho u \\ 
\rho u^2 + p \\
(e + p)u
\end{matrix}
\right]$

$\frac{\partial F}{\partial Q} = A = P\Lambda P^{-1}$

$P=\left[
\begin{matrix}
1 & 1 & 1 \\ 
u-a & u & u+a \\
H-au & \frac{u^2}{2} & H+au
\end{matrix}
\right]$

$\Lambda = \left[
\begin{matrix}
u-a & 0 & 0 \\ 
0 & u & 0 \\
0 & 0 & u+a
\end{matrix}
\right]$

$P^{-1}=\left[
\begin{matrix}
\frac{1}{2}\left( \alpha_1 + \frac{u}{a} \right) & -\frac{1}{2}\left( \alpha_2 u + \frac{1}{a} \right) & \frac{\alpha_2}{2} \\ 
1-\alpha_1 & \alpha_2u & -\alpha_2 \\
\frac{1}{2}\left( \alpha_1 - \frac{u}{a} \right) & -\frac{1}{2}\left( \alpha_2 u - \frac{1}{a} \right) & \frac{\alpha_2}{2} \\ 
\end{matrix}
\right]$

$\alpha_1 = 
\frac{\left( \gamma - 1 \right) u^2}
{2a^2}$

$\alpha_2 = 
\frac{\left( \gamma - 1 \right)}
{a^2}$

In [11]:
# constant
L=3 # [m]
U=5 # [m/s]
P=102315 # [Pa]
Rho = 0.998 # [m^3/lg]
T=298 # [K]

In [3]:
def scaler_d2n(Q):
    non_Q = np.zeros_like(Q)
    non_Q[:, 0] = Q[:, 0] / Rho
    non_Q[:, 1] = Q[:, 1] / ( Rho * U )
    non_Q[:, 2] = Q[:, 2] / ( Rho * ( U ** 2 ) )
    return non_Q

def scaler_n2d(Q):
    dim_Q = np.zeros_like(Q)
    dim_Q[:, 0] = Q[:, 0] * Rho
    dim_Q[:, 1] = Q[:, 1] * ( Rho * U )
    dim_Q[:, 2] = Q[:, 2] * ( Rho * ( U ** 2 ) )
    return dim_Q



In [7]:
# parameter
n = 1000
gamma = 1.4
g1 = gamma - 1

In [16]:
non_P = P / (Rho * U ** 2)
non_e = non_P / g1

def e2p(Q):
    rho = Q[:, 0]
    u = Q[:, 1] / Q[:, 0]
    e = Q[:, 2]
    p = g1 * ( e - 1/2 * rho * u**2 )
    return p

def e2T(Q):
    p = e2p(Q)
    rho = Q[:, 0]
    T = p / rho
    return T

In [18]:
non_L = 1
non_U = 1
non_Rho = 1
non_T = 1

In [19]:
# initial condition
Q = np.zeros((n, 3), np.float64)
Q[:, 0] = non_Rho
Q[:, 1] = non_Rho * 0
Q[:, 2] = non_e

In [23]:
dx = non_L / n
x_c = np.zeros(n)
x_e = np.zeros(n+1)
for i in range(n+1):
    x_e[i] = i * dx
    
for i in range(n):
    x_c[i] = (i + 1 / 2 ) * dx
        
print(x_c)


[5.000e-04 1.500e-03 2.500e-03 3.500e-03 4.500e-03 5.500e-03 6.500e-03
 7.500e-03 8.500e-03 9.500e-03 1.050e-02 1.150e-02 1.250e-02 1.350e-02
 1.450e-02 1.550e-02 1.650e-02 1.750e-02 1.850e-02 1.950e-02 2.050e-02
 2.150e-02 2.250e-02 2.350e-02 2.450e-02 2.550e-02 2.650e-02 2.750e-02
 2.850e-02 2.950e-02 3.050e-02 3.150e-02 3.250e-02 3.350e-02 3.450e-02
 3.550e-02 3.650e-02 3.750e-02 3.850e-02 3.950e-02 4.050e-02 4.150e-02
 4.250e-02 4.350e-02 4.450e-02 4.550e-02 4.650e-02 4.750e-02 4.850e-02
 4.950e-02 5.050e-02 5.150e-02 5.250e-02 5.350e-02 5.450e-02 5.550e-02
 5.650e-02 5.750e-02 5.850e-02 5.950e-02 6.050e-02 6.150e-02 6.250e-02
 6.350e-02 6.450e-02 6.550e-02 6.650e-02 6.750e-02 6.850e-02 6.950e-02
 7.050e-02 7.150e-02 7.250e-02 7.350e-02 7.450e-02 7.550e-02 7.650e-02
 7.750e-02 7.850e-02 7.950e-02 8.050e-02 8.150e-02 8.250e-02 8.350e-02
 8.450e-02 8.550e-02 8.650e-02 8.750e-02 8.850e-02 8.950e-02 9.050e-02
 9.150e-02 9.250e-02 9.350e-02 9.450e-02 9.550e-02 9.650e-02 9.750e-02
 9.850

$ 
        Q_i^{n+1} = 
    \frac{\triangle t}{\triangle x_i^{n+1}} \left(
        F_{i + \frac{1}{2}}^{ n }
      - F_{i - \frac{1}{2}}^{ n }
    \right)
    + Q_{i}^{n} 
    = \frac{\triangle t}{\triangle x_i^{n+1} }
    \left[ 
        \frac{1}{2} 
        \left\{ 
            \left( F_{i+1}^{n} + F_{i}^{n} \right)
          + \left| A^{roe} \right| \left( Q_{i+1}^{n} - Q_{i}^{n} \right)
        \right\} 
        - \frac{1}{2} 
            \left( F_{i}^{n} + F_{i-1}^{n} \right)
          + \left| A^{roe} \right| \left( Q_{i}^{n} - Q_{i-1}^{n} \right)
    \right]
    + Q_{i}^{n}
$


In [ ]:
F = np.zeros_like(Q)
A = np.zeros((n, 3, 3), np.float64)

def flux(Q):
    rho = Q[:, 0]
    rho_u = Q[:, 1]
    u = rho_u / rho
    e = Q[:, 2]
    p = e2p(Q)
    
    F[:, 0] = rho_u
    F[:, 1] = rho_u * u + p
    F[:, 2] = (e + p) * u
    return F
    
def roe_matrix(Q):
    
    

def rhs(Q):
    F = flux(Q)
    